# Day 23: 持久化弹性 Agent - Google ADK + Restate

## 学习目标

1. 理解为什么 Agent 需要持久化执行
2. 掌握 Restate 的核心概念
3. 学会使用 Google ADK + Restate 构建弹性 Agent
4. 实践人工审批等长时间等待场景

## 1. 问题：传统 Agent 的脆弱性

让我们先模拟一个传统 Agent 的问题。

In [1]:
import random
import time

class FragileClaimsAgent:
    """一个脆弱的传统 Agent 实现"""
    
    def __init__(self):
        self.llm_calls = 0
        self.total_cost = 0.0
    
    def call_llm(self, prompt: str) -> str:
        """模拟 LLM 调用（每次都有成本）"""
        self.llm_calls += 1
        self.total_cost += 0.01  # 假设每次调用 $0.01
        time.sleep(0.5)  # 模拟延迟
        print(f"  📞 LLM 调用 #{self.llm_calls} (累计成本: ${self.total_cost:.2f})")
        return f"分析结果_{self.llm_calls}"
    
    def process_claim(self, claim_data: dict, fail_probability: float = 0.3):
        """处理理赔申请 - 可能在任何步骤失败"""
        print(f"\n🚀 开始处理理赔: {claim_data['id']}")
        
        # Step 1: 分析理赔
        print("Step 1: 分析理赔...")
        analysis = self.call_llm(f"分析理赔 {claim_data}")
        
        if random.random() < fail_probability:
            raise RuntimeError("💥 网络错误！进程崩溃！")
        
        # Step 2: 验证文档
        print("Step 2: 验证文档...")
        verification = self.call_llm(f"验证文档 {claim_data['documents']}")
        
        if random.random() < fail_probability:
            raise RuntimeError("💥 内存不足！进程崩溃！")
        
        # Step 3: 风险评估
        print("Step 3: 风险评估...")
        risk = self.call_llm(f"评估风险 {analysis} {verification}")
        
        if random.random() < fail_probability:
            raise RuntimeError("💥 服务超时！进程崩溃！")
        
        # Step 4: 生成报告
        print("Step 4: 生成报告...")
        report = self.call_llm(f"生成报告 {risk}")
        
        print(f"\n✅ 处理完成！")
        return {
            "status": "completed",
            "analysis": analysis,
            "verification": verification,
            "risk": risk,
            "report": report
        }

In [2]:
# 模拟处理一个理赔申请（很可能失败）
random.seed(42)  # 固定随机种子以便复现

claim = {
    "id": "CLM-001",
    "amount": 5000,
    "documents": ["receipt.pdf", "photo.jpg"]
}

agent = FragileClaimsAgent()

# 尝试多次直到成功
max_attempts = 5
for attempt in range(1, max_attempts + 1):
    print(f"\n{'='*50}")
    print(f"尝试 #{attempt}")
    print(f"{'='*50}")
    
    try:
        result = agent.process_claim(claim, fail_probability=0.4)
        print(f"\n🎉 最终成功！")
        break
    except RuntimeError as e:
        print(f"\n{e}")
        print(f"⚠️  需要重新开始...所有之前的 LLM 调用都浪费了！")

print(f"\n📊 统计:")
print(f"   - 总 LLM 调用次数: {agent.llm_calls}")
print(f"   - 总成本: ${agent.total_cost:.2f}")
print(f"   - 如果使用持久化，最少只需 4 次调用 (${0.04:.2f})")


尝试 #1

🚀 开始处理理赔: CLM-001
Step 1: 分析理赔...
  📞 LLM 调用 #1 (累计成本: $0.01)
Step 2: 验证文档...
  📞 LLM 调用 #2 (累计成本: $0.02)

💥 内存不足！进程崩溃！
⚠️  需要重新开始...所有之前的 LLM 调用都浪费了！

尝试 #2

🚀 开始处理理赔: CLM-001
Step 1: 分析理赔...
  📞 LLM 调用 #3 (累计成本: $0.03)

💥 网络错误！进程崩溃！
⚠️  需要重新开始...所有之前的 LLM 调用都浪费了！

尝试 #3

🚀 开始处理理赔: CLM-001
Step 1: 分析理赔...
  📞 LLM 调用 #4 (累计成本: $0.04)

💥 网络错误！进程崩溃！
⚠️  需要重新开始...所有之前的 LLM 调用都浪费了！

尝试 #4

🚀 开始处理理赔: CLM-001
Step 1: 分析理赔...
  📞 LLM 调用 #5 (累计成本: $0.05)
Step 2: 验证文档...
  📞 LLM 调用 #6 (累计成本: $0.06)
Step 3: 风险评估...
  📞 LLM 调用 #7 (累计成本: $0.07)
Step 4: 生成报告...
  📞 LLM 调用 #8 (累计成本: $0.08)

✅ 处理完成！

🎉 最终成功！

📊 统计:
   - 总 LLM 调用次数: 8
   - 总成本: $0.08
   - 如果使用持久化，最少只需 4 次调用 ($0.04)


## 2. 解决方案：持久化执行模式

让我们模拟 Restate 的持久化执行模式。

In [3]:
import json
from typing import Any, Callable, Optional
from dataclasses import dataclass, field

@dataclass
class DurableJournal:
    """模拟 Restate 的持久化日志"""
    entries: dict = field(default_factory=dict)
    
    def has(self, step_name: str) -> bool:
        return step_name in self.entries
    
    def get(self, step_name: str) -> Any:
        return self.entries[step_name]
    
    def set(self, step_name: str, value: Any):
        self.entries[step_name] = value
        # 在真实实现中，这会持久化到磁盘/数据库
        print(f"  💾 持久化: {step_name}")
    
    def save_to_file(self, path: str):
        with open(path, 'w') as f:
            json.dump(self.entries, f)
    
    @classmethod
    def load_from_file(cls, path: str) -> 'DurableJournal':
        try:
            with open(path) as f:
                return cls(entries=json.load(f))
        except FileNotFoundError:
            return cls()


class DurableContext:
    """模拟 Restate 的 Context API"""
    
    def __init__(self, journal: DurableJournal):
        self.journal = journal
        self.replayed_steps = 0
        self.executed_steps = 0
    
    async def run(self, step_name: str, fn: Callable) -> Any:
        """执行一个可持久化的步骤"""
        if self.journal.has(step_name):
            # 从日志恢复，不需要重新执行
            self.replayed_steps += 1
            result = self.journal.get(step_name)
            print(f"  ♻️  重放: {step_name} (从日志恢复)")
            return result
        else:
            # 执行并持久化
            self.executed_steps += 1
            result = fn()
            self.journal.set(step_name, result)
            return result

In [4]:
class DurableClaimsAgent:
    """使用持久化执行的 Agent"""
    
    def __init__(self):
        self.llm_calls = 0
        self.total_cost = 0.0
    
    def call_llm(self, prompt: str) -> str:
        """模拟 LLM 调用"""
        self.llm_calls += 1
        self.total_cost += 0.01
        time.sleep(0.3)
        print(f"  📞 LLM 调用 #{self.llm_calls} (成本: ${self.total_cost:.2f})")
        return f"分析结果_{self.llm_calls}"
    
    async def process_claim(self, ctx: DurableContext, claim_data: dict, fail_at_step: int = None):
        """持久化的理赔处理"""
        print(f"\n🚀 开始处理理赔: {claim_data['id']}")
        
        # Step 1: 分析理赔 (持久化)
        print("Step 1: 分析理赔...")
        analysis = await ctx.run(
            "analyze_claim",
            lambda: self.call_llm(f"分析理赔 {claim_data}")
        )
        
        if fail_at_step == 1:
            raise RuntimeError("💥 模拟崩溃 @ Step 1")
        
        # Step 2: 验证文档 (持久化)
        print("Step 2: 验证文档...")
        verification = await ctx.run(
            "verify_documents",
            lambda: self.call_llm(f"验证文档 {claim_data['documents']}")
        )
        
        if fail_at_step == 2:
            raise RuntimeError("💥 模拟崩溃 @ Step 2")
        
        # Step 3: 风险评估 (持久化)
        print("Step 3: 风险评估...")
        risk = await ctx.run(
            "assess_risk",
            lambda: self.call_llm(f"评估风险 {analysis} {verification}")
        )
        
        if fail_at_step == 3:
            raise RuntimeError("💥 模拟崩溃 @ Step 3")
        
        # Step 4: 生成报告 (持久化)
        print("Step 4: 生成报告...")
        report = await ctx.run(
            "generate_report",
            lambda: self.call_llm(f"生成报告 {risk}")
        )
        
        print(f"\n✅ 处理完成！")
        return {
            "status": "completed",
            "analysis": analysis,
            "verification": verification,
            "risk": risk,
            "report": report
        }

In [5]:
import asyncio

async def demo_durable_execution():
    """演示持久化执行"""
    claim = {
        "id": "CLM-002",
        "amount": 5000,
        "documents": ["receipt.pdf", "photo.jpg"]
    }
    
    # 创建持久化日志
    journal = DurableJournal()
    agent = DurableClaimsAgent()
    
    # 第一次尝试：在 Step 2 后崩溃
    print("="*60)
    print("第一次执行 (将在 Step 2 后崩溃)")
    print("="*60)
    
    ctx = DurableContext(journal)
    try:
        await agent.process_claim(ctx, claim, fail_at_step=2)
    except RuntimeError as e:
        print(f"\n{e}")
        print(f"\n📊 第一次执行统计:")
        print(f"   - LLM 调用: {agent.llm_calls}")
        print(f"   - 已持久化步骤: {list(journal.entries.keys())}")
    
    # 第二次尝试：恢复执行（不会在 Step 2 崩溃）
    print("\n" + "="*60)
    print("第二次执行 (恢复，将在 Step 3 后崩溃)")
    print("="*60)
    
    ctx = DurableContext(journal)  # 使用同一个 journal
    try:
        await agent.process_claim(ctx, claim, fail_at_step=3)
    except RuntimeError as e:
        print(f"\n{e}")
        print(f"\n📊 第二次执行统计:")
        print(f"   - 新增 LLM 调用: {agent.llm_calls - 2}")
        print(f"   - 重放步骤: {ctx.replayed_steps}")
        print(f"   - 已持久化步骤: {list(journal.entries.keys())}")
    
    # 第三次尝试：完成执行
    print("\n" + "="*60)
    print("第三次执行 (恢复并完成)")
    print("="*60)
    
    ctx = DurableContext(journal)
    result = await agent.process_claim(ctx, claim)  # 不会崩溃
    
    print(f"\n📊 最终统计:")
    print(f"   - 总 LLM 调用: {agent.llm_calls}")
    print(f"   - 总成本: ${agent.total_cost:.2f}")
    print(f"   - 如果没有持久化，至少需要 12 次调用 (${0.12:.2f})")
    print(f"   - 节省: ${0.12 - agent.total_cost:.2f} ({(1 - agent.total_cost/0.12)*100:.0f}%)")

# 运行演示
await demo_durable_execution()

第一次执行 (将在 Step 2 后崩溃)

🚀 开始处理理赔: CLM-002
Step 1: 分析理赔...
  📞 LLM 调用 #1 (成本: $0.01)
  💾 持久化: analyze_claim
Step 2: 验证文档...
  📞 LLM 调用 #2 (成本: $0.02)
  💾 持久化: verify_documents

💥 模拟崩溃 @ Step 2

📊 第一次执行统计:
   - LLM 调用: 2
   - 已持久化步骤: ['analyze_claim', 'verify_documents']

第二次执行 (恢复，将在 Step 3 后崩溃)

🚀 开始处理理赔: CLM-002
Step 1: 分析理赔...
  ♻️  重放: analyze_claim (从日志恢复)
Step 2: 验证文档...
  ♻️  重放: verify_documents (从日志恢复)
Step 3: 风险评估...
  📞 LLM 调用 #3 (成本: $0.03)
  💾 持久化: assess_risk

💥 模拟崩溃 @ Step 3

📊 第二次执行统计:
   - 新增 LLM 调用: 1
   - 重放步骤: 2
   - 已持久化步骤: ['analyze_claim', 'verify_documents', 'assess_risk']

第三次执行 (恢复并完成)

🚀 开始处理理赔: CLM-002
Step 1: 分析理赔...
  ♻️  重放: analyze_claim (从日志恢复)
Step 2: 验证文档...
  ♻️  重放: verify_documents (从日志恢复)
Step 3: 风险评估...
  ♻️  重放: assess_risk (从日志恢复)
Step 4: 生成报告...
  📞 LLM 调用 #4 (成本: $0.04)
  💾 持久化: generate_report

✅ 处理完成！

📊 最终统计:
   - 总 LLM 调用: 4
   - 总成本: $0.04
   - 如果没有持久化，至少需要 12 次调用 ($0.12)
   - 节省: $0.08 (67%)


## 3. 人工审批等待场景

Restate 最强大的功能之一是支持长时间等待（如人工审批），进程可以终止，状态保持。

In [6]:
import uuid
from datetime import datetime
from typing import Optional
from dataclasses import dataclass

@dataclass
class Awakeable:
    """模拟 Restate 的 Awakeable（可等待的 Promise）"""
    id: str
    resolved: bool = False
    result: Optional[Any] = None

# 全局存储（模拟 Restate Server）
_awakeables: dict[str, Awakeable] = {}

class EnhancedDurableContext(DurableContext):
    """增强的 Context，支持 Awakeable"""
    
    def awakeable(self) -> Awakeable:
        """创建一个可等待的 Promise"""
        awakeable_id = f"awk_{uuid.uuid4().hex[:8]}"
        awk = Awakeable(id=awakeable_id)
        _awakeables[awakeable_id] = awk
        print(f"  🔔 创建 Awakeable: {awakeable_id}")
        return awk
    
    async def wait_for_awakeable(self, awk: Awakeable, timeout_message: str = None):
        """等待 Awakeable 被触发"""
        print(f"  ⏳ 等待外部触发: {awk.id}")
        if timeout_message:
            print(f"     ({timeout_message})")
        
        # 在真实实现中，这里会暂停执行
        # 进程可以终止，等触发后再恢复
        while not awk.resolved:
            await asyncio.sleep(0.1)  # 模拟等待
        
        print(f"  ✅ Awakeable 已触发: {awk.id}")
        return awk.result


def resolve_awakeable(awakeable_id: str, result: Any):
    """外部调用：触发 Awakeable（模拟人工审批完成）"""
    if awakeable_id in _awakeables:
        awk = _awakeables[awakeable_id]
        awk.resolved = True
        awk.result = result
        print(f"\n🔔 外部触发 Awakeable: {awakeable_id}")
        print(f"   结果: {result}")

In [7]:
class ClaimsAgentWithApproval:
    """需要人工审批的理赔 Agent"""
    
    def __init__(self):
        self.llm_calls = 0
    
    def call_llm(self, prompt: str) -> str:
        self.llm_calls += 1
        time.sleep(0.2)
        return f"结果_{self.llm_calls}"
    
    async def process_claim_with_approval(
        self, 
        ctx: EnhancedDurableContext, 
        claim_data: dict
    ):
        """需要人工审批的理赔处理流程"""
        print(f"\n🚀 开始处理理赔: {claim_data['id']}")
        print(f"   金额: ${claim_data['amount']}")
        
        # Step 1: AI 分析
        print("\nStep 1: AI 分析...")
        analysis = await ctx.run(
            "analyze",
            lambda: self.call_llm(f"分析 {claim_data}")
        )
        print(f"   分析结果: {analysis}")
        
        # Step 2: 风险评估
        print("\nStep 2: 风险评估...")
        risk_score = await ctx.run(
            "risk_assessment",
            lambda: 0.7 if claim_data['amount'] > 3000 else 0.2
        )
        print(f"   风险分数: {risk_score}")
        
        # Step 3: 高风险需要人工审批
        if risk_score > 0.5:
            print("\nStep 3: 需要人工审批 (高风险)")
            print("="*40)
            print("📧 已发送审批请求到审批人")
            print("   进程可以在此终止，等待审批...")
            print("="*40)
            
            # 创建可等待的 Promise
            approval_awakeable = ctx.awakeable()
            
            # 模拟发送通知
            await ctx.run(
                "send_approval_request",
                lambda: f"已通知审批人，回调 ID: {approval_awakeable.id}"
            )
            
            # 等待人工审批（可能需要数小时或数天）
            approval_result = await ctx.wait_for_awakeable(
                approval_awakeable,
                timeout_message="等待人工审批中，可能需要 1-3 个工作日"
            )
            
            if not approval_result.get("approved"):
                return {
                    "status": "rejected",
                    "reason": approval_result.get("reason", "审批拒绝")
                }
        else:
            print("\nStep 3: 自动审批 (低风险)")
        
        # Step 4: 处理支付
        print("\nStep 4: 处理支付...")
        payment = await ctx.run(
            "process_payment",
            lambda: {
                "transaction_id": f"TXN_{uuid.uuid4().hex[:8]}",
                "amount": claim_data['amount'],
                "status": "completed"
            }
        )
        print(f"   支付完成: {payment['transaction_id']}")
        
        return {
            "status": "completed",
            "analysis": analysis,
            "risk_score": risk_score,
            "payment": payment
        }

In [8]:
async def demo_human_approval():
    """演示人工审批场景"""
    
    # 高额理赔（需要人工审批）
    high_value_claim = {
        "id": "CLM-HIGH-001",
        "amount": 10000,
        "documents": ["medical_report.pdf", "receipt.pdf"]
    }
    
    journal = DurableJournal()
    ctx = EnhancedDurableContext(journal)
    agent = ClaimsAgentWithApproval()
    
    # 创建处理任务
    claim_task = asyncio.create_task(
        agent.process_claim_with_approval(ctx, high_value_claim)
    )
    
    # 模拟：等待一段时间后，人工审批完成
    await asyncio.sleep(1)  # 在真实场景中可能是几天
    
    # 获取 awakeable ID（在真实场景中，这会通过 API 调用）
    awakeable_id = list(_awakeables.keys())[-1]
    
    print("\n" + "="*60)
    print("👤 审批人操作面板")
    print("="*60)
    print(f"理赔 ID: {high_value_claim['id']}")
    print(f"金额: ${high_value_claim['amount']}")
    print(f"Awakeable ID: {awakeable_id}")
    print("\n正在审批...")
    
    # 模拟人工审批
    await asyncio.sleep(0.5)
    resolve_awakeable(awakeable_id, {
        "approved": True,
        "approver": "张经理",
        "timestamp": datetime.now().isoformat()
    })
    
    # 等待任务完成
    result = await claim_task
    
    print("\n" + "="*60)
    print("📋 最终结果")
    print("="*60)
    print(f"状态: {result['status']}")
    if 'payment' in result:
        print(f"支付交易: {result['payment']['transaction_id']}")

# 运行演示
_awakeables.clear()  # 清理之前的状态
await demo_human_approval()


🚀 开始处理理赔: CLM-HIGH-001
   金额: $10000

Step 1: AI 分析...
  💾 持久化: analyze
   分析结果: 结果_1

Step 2: 风险评估...
  💾 持久化: risk_assessment
   风险分数: 0.7

Step 3: 需要人工审批 (高风险)
📧 已发送审批请求到审批人
   进程可以在此终止，等待审批...
  🔔 创建 Awakeable: awk_c4bfbb33
  💾 持久化: send_approval_request
  ⏳ 等待外部触发: awk_c4bfbb33
     (等待人工审批中，可能需要 1-3 个工作日)

👤 审批人操作面板
理赔 ID: CLM-HIGH-001
金额: $10000
Awakeable ID: awk_c4bfbb33

正在审批...

🔔 外部触发 Awakeable: awk_c4bfbb33
   结果: {'approved': True, 'approver': '张经理', 'timestamp': '2025-12-31T16:57:35.977790'}
  ✅ Awakeable 已触发: awk_c4bfbb33

Step 4: 处理支付...
  💾 持久化: process_payment
   支付完成: TXN_00773ae6

📋 最终结果
状态: completed
支付交易: TXN_00773ae6


## 4. Restate 真实代码示例

以下是使用真实 Restate SDK 的代码结构。

In [9]:
# 以下是真实的 Restate + Google ADK 代码（需要安装依赖才能运行）

restate_code = '''
# claims_service.py
from restate import Service, Context
from google.adk import Agent, tool

# 定义 Google ADK Agent
@tool
def analyze_claim(claim_data: dict) -> dict:
    """使用 AI 分析理赔申请"""
    return {"risk_score": 0.3, "recommendation": "approve"}

@tool
def process_payment(claim_id: str, amount: float) -> dict:
    """处理支付"""
    return {"transaction_id": "TXN123", "status": "completed"}

claims_agent = Agent(
    name="ClaimsProcessor",
    model="gemini-2.0-flash",
    instruction="你是一个保险理赔处理专家。",
    tools=[analyze_claim, process_payment]
)

# 创建 Restate 服务
claims_service = Service("claims")

@claims_service.handler()
async def process_claim(ctx: Context, claim_data: dict) -> dict:
    """
    持久化的理赔处理工作流
    """
    # Step 1: AI 分析 (持久化)
    analysis = await ctx.run(
        "analyze_claim",
        lambda: claims_agent.tools.analyze_claim(claim_data)
    )
    
    # Step 2: 等待人工审批 (可暂停数天)
    if analysis["risk_score"] > 0.5:
        approval_promise = ctx.awakeable()
        await ctx.run(
            "notify_reviewer",
            lambda: send_notification(approval_promise.id)
        )
        
        # 暂停执行，等待外部触发
        approval = await approval_promise
        
        if not approval["approved"]:
            return {"status": "rejected"}
    
    # Step 3: 处理支付 (持久化)
    payment = await ctx.run(
        "process_payment",
        lambda: claims_agent.tools.process_payment(
            claim_data["id"],
            claim_data["amount"]
        )
    )
    
    return {"status": "completed", "payment": payment}


# main.py - 启动服务
from restate import app

app = restate.app(services=[claims_service])

if __name__ == "__main__":
    app.run()
'''

print("📝 Restate + Google ADK 真实代码示例:")
print(restate_code)

📝 Restate + Google ADK 真实代码示例:

# claims_service.py
from restate import Service, Context
from google.adk import Agent, tool

# 定义 Google ADK Agent
@tool
def analyze_claim(claim_data: dict) -> dict:
    """使用 AI 分析理赔申请"""
    return {"risk_score": 0.3, "recommendation": "approve"}

@tool
def process_payment(claim_id: str, amount: float) -> dict:
    """处理支付"""
    return {"transaction_id": "TXN123", "status": "completed"}

claims_agent = Agent(
    name="ClaimsProcessor",
    model="gemini-2.0-flash",
    instruction="你是一个保险理赔处理专家。",
    tools=[analyze_claim, process_payment]
)

# 创建 Restate 服务
claims_service = Service("claims")

@claims_service.handler()
async def process_claim(ctx: Context, claim_data: dict) -> dict:
    """
    持久化的理赔处理工作流
    """
    # Step 1: AI 分析 (持久化)
    analysis = await ctx.run(
        "analyze_claim",
        lambda: claims_agent.tools.analyze_claim(claim_data)
    )

    # Step 2: 等待人工审批 (可暂停数天)
    if analysis["risk_score"] > 0.5:
        approval_promise 

## 5. 运行官方示例

要运行官方示例，请按以下步骤操作：

In [10]:
setup_instructions = """
# 快速开始：运行官方示例

## 1. 克隆示例仓库
git clone https://github.com/restatedev/restate-google-adk-example
cd restate-google-adk-example

## 2. 设置 API Key
export GOOGLE_API_KEY="your-key-here"

## 3. 启动 Agent（使用 uv）
uv run .

## 4. 在新终端中启动 Restate Server
docker run --name restate --rm \
  -p 8080:8080 \
  -p 9070:9070 \
  --add-host host.docker.internal:host-gateway \
  docker.restate.dev/restatedev/restate:latest

## 5. 打开 Restate UI
# 访问 http://localhost:9070

## 6. 发送测试请求
curl -X POST http://localhost:8080/claims/process_claim \
  -H "Content-Type: application/json" \
  -d '{"id": "CLM-001", "amount": 5000}'
"""

print(setup_instructions)


# 快速开始：运行官方示例

## 1. 克隆示例仓库
git clone https://github.com/restatedev/restate-google-adk-example
cd restate-google-adk-example

## 2. 设置 API Key
export GOOGLE_API_KEY="your-key-here"

## 3. 启动 Agent（使用 uv）
uv run .

## 4. 在新终端中启动 Restate Server
docker run --name restate --rm   -p 8080:8080   -p 9070:9070   --add-host host.docker.internal:host-gateway   docker.restate.dev/restatedev/restate:latest

## 5. 打开 Restate UI
# 访问 http://localhost:9070

## 6. 发送测试请求
curl -X POST http://localhost:8080/claims/process_claim   -H "Content-Type: application/json"   -d '{"id": "CLM-001", "amount": 5000}'



## 6. 关键概念总结

In [11]:
summary = """
╔══════════════════════════════════════════════════════════════════╗
║              Day 23 关键概念总结                                   ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  1. 问题：传统 Agent 的脆弱性                                     ║
║     • 崩溃后丢失所有状态                                          ║
║     • 重试需要重复所有 LLM 调用（浪费成本）                         ║
║     • 无法处理长时间等待（如人工审批）                              ║
║                                                                  ║
║  2. 解决方案：Restate 持久化执行                                   ║
║     • Durable Journal: 记录每个步骤的结果                         ║
║     • Replay: 恢复时从日志读取而非重新执行                         ║
║     • Awakeable: 可跨重启的 Promise                              ║
║                                                                  ║
║  3. 核心 API                                                     ║
║     • ctx.run(name, fn): 持久化执行                              ║
║     • ctx.awakeable(): 创建可等待的 Promise                       ║
║     • ctx.sleep(): 持久化休眠                                    ║
║     • ctx.call(): 调用其他服务                                   ║
║                                                                  ║
║  4. 最佳实践                                                      ║
║     • 使用确定性 ID（幂等性）                                      ║
║     • 细粒度步骤划分                                              ║
║     • 正确的错误处理                                              ║
║                                                                  ║
║  5. 架构优势                                                      ║
║     • Google ADK: Agent 逻辑 + LLM 编排                          ║
║     • Restate: 状态管理 + 持久化 + 恢复                           ║
║     • 组合: 弹性、可扩展的 AI Agent                               ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
"""

print(summary)


╔══════════════════════════════════════════════════════════════════╗
║              Day 23 关键概念总结                                   ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  1. 问题：传统 Agent 的脆弱性                                     ║
║     • 崩溃后丢失所有状态                                          ║
║     • 重试需要重复所有 LLM 调用（浪费成本）                         ║
║     • 无法处理长时间等待（如人工审批）                              ║
║                                                                  ║
║  2. 解决方案：Restate 持久化执行                                   ║
║     • Durable Journal: 记录每个步骤的结果                         ║
║     • Replay: 恢复时从日志读取而非重新执行                         ║
║     • Awakeable: 可跨重启的 Promise                              ║
║                                                                  ║
║  3. 核心 API                                                     ║
║     • ctx.run(name, fn): 持久化执行             

## 7. 练习题

In [12]:
exercises = """
📝 练习题

1. 基础练习
   修改 DurableClaimsAgent，添加第 5 步「发送通知」，
   并确保它也是持久化的。

2. 中级练习
   实现一个 retry 机制：如果某个步骤失败（如 LLM 调用超时），
   自动重试最多 3 次，每次间隔递增。

3. 高级练习
   设计一个多级审批流程：
   - 金额 < $1000: 自动审批
   - $1000-$5000: 需要一级审批
   - > $5000: 需要一级和二级审批（串行）
   
   使用 Awakeable 实现等待多个审批。

4. 思考题
   - 为什么 ctx.run() 中的函数应该是幂等的？
   - 如果审批人一周都没有响应怎么办？如何设计超时机制？
   - Restate 的 Durable Journal 和数据库事务有什么区别？
"""

print(exercises)


📝 练习题

1. 基础练习
   修改 DurableClaimsAgent，添加第 5 步「发送通知」，
   并确保它也是持久化的。

2. 中级练习
   实现一个 retry 机制：如果某个步骤失败（如 LLM 调用超时），
   自动重试最多 3 次，每次间隔递增。

3. 高级练习
   设计一个多级审批流程：
   - 金额 < $1000: 自动审批
   - $1000-$5000: 需要一级审批
   - > $5000: 需要一级和二级审批（串行）

   使用 Awakeable 实现等待多个审批。

4. 思考题
   - 为什么 ctx.run() 中的函数应该是幂等的？
   - 如果审批人一周都没有响应怎么办？如何设计超时机制？
   - Restate 的 Durable Journal 和数据库事务有什么区别？



## 参考资源

- [Restate 官方文档](https://docs.restate.dev)
- [Restate AI Agent 指南](https://docs.restate.dev/ai)
- [Google ADK 文档](https://google.github.io/adk-docs/)
- [示例仓库](https://github.com/restatedev/restate-google-adk-example)
- [Restate GitHub](https://github.com/restatedev/restate)
- [关于 Serverless + Durable Execution 的博客](https://www.restate.dev/blog/resilient-serverless-agents)